# Invisible cloak using OpenCV

The key idea is to replace the current frame pixels corresponding to the cloth/cloak 
with the background pixels to generate the effect of an invisibility cloak.

In [ ]:
# Install the opencv library, if not installed
# pip install opencv-python

## 1. Import the libraries

In [ ]:
import cv2          # OpenCV library
import numpy as np  # Numpy library used to deal with the arrays
import time         # Time library to use sleep()

## 2. Record/Upload video and store background for each frame

In [ ]:
# Read from the video, insert your video title instead of video.mp4
# Incase you want to read from webcam use cv2.VideoCapture(0)
input_video = cv2.VideoCapture("video.mp4")

In [ ]:
# Let the webcam start by giving it 2 seconds
time.sleep(2) 
count = 0 
background = 0 

In [ ]:
# Capturing the background in range of 30 (depends on your video)
for i in range(30):
	rval , background = input_video.read()
	if rval == False :
		continue 
# Laterally invert the image / flip the image
background = np.flip(background,axis=1)

## 3. Detect the defined color portion and replace with the mask image in each frame

In [ ]:
# We read every frame from the video/webcam, until the video is open 
while (input_video.isOpened()):
	rval, img = input_video.read()
	if not rval :
		break 
	count = count + 1
	img = np.flip(img , axis=1)
    
	# Convert the image from BGR(Blue, Green, Red) to HSV(Hue, Saturation, Value) 
	hsv = cv2.cvtColor(img , cv2.COLOR_BGR2HSV)
    
	# Generate mask to detect red color
    # You can choose any other color cloth also, you just need to change the HSV values.
    
    # Range for lower red
	lower_red = np.array([100, 40, 40])
	upper_red = np.array([100, 255, 255])
	mask1 = cv2.inRange(hsv,lower_red,upper_red)

    # Range for upper red
	lower_red = np.array([155, 40, 40])
	upper_red = np.array([180, 255, 255])
	mask2 = cv2.inRange(hsv,lower_red,upper_red)
    
    # Using the below line, we combine masks generated for both the red color range.
	mask1 = mask1+mask2

    # Refining the mask corresponding to the detected red color
    # Open and dilate the mask image
	mask1 = cv2.morphologyEx(mask1, cv2.MORPH_OPEN, np.ones((3,3),np.uint8),iterations=2)
	mask1 = cv2.dilate(mask1,np.ones((3,3),np.uint8),iterations = 1)
    
    # Create an inverted mask to segment out the red color from the frame
	mask2 = cv2.bitwise_not(mask1)

    # Segment the red color part out of the frame using bitwise and with the inverted mask
	res1 = cv2.bitwise_and(background,background,mask=mask1)
    
    # Create image showing static background frame pixels only for the masked region
	res2 = cv2.bitwise_and(img,img,mask=mask2)
    
    # Generating the final output
	output_video = cv2.addWeighted(res1,1,res2,1,0)
    
    # Show the output video with title 'Let's be invisble'
	cv2.imshow("Let's be invisble",output_video)
	k = cv2.waitKey(10)
	if k == 27: #esc key
		break

In [ ]:
# Close your video window
input_video.release()
cv2.destroyAllWindows()